In [25]:
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import seaborn as sns
import shap
from pandas import json_normalize
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             ndcg_score, r2_score)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [26]:
MAX_SEASON = 2022
LAST_N_SEASONS = 3
RANDOM_SEED = 12345
random.seed(RANDOM_SEED)

COLLECTION_PLAYER = "player"
COLLECTION_TEAM = "team"

In [51]:
df = pd.read_csv("data.csv", encoding="utf-8-sig")


'''
df.drop(["team_id", "champion"],
        axis="columns",  
        inplace=True)
df['Rk_Conference'] = df.groupby(['season', 'conference'])['W'].rank("min", ascending=False)
df['Rk_Conference'] = df['Rk_Conference'] + df['Rk_Season']
df['Rk_Conference'] = df.groupby(['season', 'conference'])['Rk_Conference'].rank("min", ascending=True)
del df['conference']

df['Top_3_Conference'] = df['Rk_Conference'].apply(lambda cell: True if cell <=3 else False)
df.to_csv("data_edit.csv", index=False, encoding="utf-8-sig")
'''
df = df[df['season'] <= MAX_SEASON]
print(df)

                                                team_id  season conference   
30    https://www.basketball-reference.com/teams/DAL...    2022       West  \
31    https://www.basketball-reference.com/teams/PHI...    2022       East   
32    https://www.basketball-reference.com/teams/HOU...    2022       West   
33    https://www.basketball-reference.com/teams/TOR...    2022       East   
34    https://www.basketball-reference.com/teams/NOP...    2022       West   
...                                                 ...     ...        ...   
1598  https://www.basketball-reference.com/teams/MNL...    1950       East   
1599  https://www.basketball-reference.com/teams/SYR...    1950       East   
1600  https://www.basketball-reference.com/teams/INO...    1950       West   
1601  https://www.basketball-reference.com/teams/AND...    1950       West   
1602  https://www.basketball-reference.com/teams/BLB...    1950       East   

        FG   FGA    FG%    3P   3PA    3P%    2P  ...    L1YP  

In [52]:
df = df.drop(['conference', 'team_id','name'], axis=1)
df = df.dropna()

print(df)

      season    FG   FGA    FG%    3P   3PA    3P%    2P   2PA    2P%  ...   
30      2022  39.3  85.1  0.461  13.1  37.4  0.350  26.2  47.8  0.548  ...  \
31      2022  39.4  84.5  0.466  11.6  31.8  0.364  27.8  52.7  0.527  ...   
32      2022  39.4  86.4  0.456  13.5  38.7  0.349  25.9  47.7  0.543  ...   
33      2022  40.6  91.3  0.445  11.9  34.2  0.349  28.7  57.1  0.503  ...   
34      2022  40.2  88.0  0.457  10.6  32.1  0.332  29.5  55.9  0.528  ...   
...      ...   ...   ...    ...   ...   ...    ...   ...   ...    ...  ...   
1219    1980  44.1  90.1  0.490   2.0   5.1  0.384  42.1  84.9  0.496  ...   
1220    1980  46.4  93.6  0.496   0.5   2.3  0.220  45.9  91.2  0.503  ...   
1221    1980  46.5  98.1  0.474   0.4   2.3  0.193  46.0  95.8  0.481  ...   
1222    1980  47.5  89.9  0.529   0.2   1.2  0.200  47.3  88.6  0.534  ...   
1223    1980  43.0  87.3  0.492   0.3   1.5  0.216  42.6  85.7  0.497  ...   

          L1YP      L3YP      L6YP     L10YP  count_playoff_gam

In [110]:
# Select the features
features = df.drop('champion', axis=1).apply(pd.to_numeric, errors='coerce').dropna().values

# Select the labels
labels = df['champion'].values

# Make sure they are not objects
features = features.astype(np.float32)
labels = labels.astype(np.float32)

eighty_percent = int(features.shape[0] * 0.8)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
print(type(X_train))
# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


<class 'numpy.ndarray'>


In [87]:
print(features.shape, labels.shape)
print(features.dtype, labels.dtype)

features.astype(np.float32)
labels.astype(np.float32)

(1165, 89) (1165,)
float32 float32


array([0., 0., 0., ..., 0., 1., 0.], dtype=float32)

In [172]:
# class TrainTest:

#     no_grad = torch.no_grad

#     def fit(self, data):
#         ## Training loop
#         self.train()        ## Set model into training mode
#         ## Iterate over the data batches
#         for batch, (inputs, target) in enumerate(data):
#             ## In real pytorch, you'd need to set the device
#             inputs = inputs.to(self.device)
#             target = target.to(self.device)
#             ## Erase the gradient history
#             self.optimizer.zero_grad()
#             ## Do a forward pass on the model
#             output = self(inputs)
#             ## Compute the loss
#             loss = self.loss(output, target)
#             ## Run backwards pass from the loss through the previous layers
#             ## This will accumulate gradients for the parameters that need to be optimized
#             loss.backward()
#             ## Perform a single optimization step
#             self.optimizer.step()
#         return {'loss' : loss}

#     def evaluate(self, data):
#         ## Set model into "evaluate" mode so that the parameters don't get updated
#         self.eval()
#         total_loss = 0
#         ## Cut off the tensor training scope to make sure weights aren't updated
#         ## For now, it's torch.no_grad; later, you'll use Tensor.no_grad
#         with TrainTest.no_grad():
#             for inputs, target in data:
#                 ## In real pytorch, you'd need to set the device
#                 inputs = inputs.to(self.device)
#                 target = target.to(self.device)
#                 output = self(inputs)
#                 total_loss += self.loss(output, target).item()  # sum up batch loss

#         total_loss /= len(data)
#         return {'test_loss' : total_loss}
        
#     def train_test(self, train_data, test_data, epochs=200):
#         ## Does both training and validation on a per-epoch basis
#         all_stats = []
#         for epoch in range(epochs):
#             train_stats = self.fit(train_data)
#             test_stats = self.evaluate(test_data)
#             all_stats += [{**train_stats, **test_stats}]
#             print(f'[Epoch {epoch+1}/{epochs}]', all_stats[-1])
#         return all_stats

# class Regression(nn.Module):
#     def __init__(self):
#         super(Regression,self).__init__()
#         self.fc1 = nn.Linear(89, 32)
#         self.fc2 = nn.Linear(32, 32)
#         self.fc3 = nn.Linear(32, 1)
# #         self.optimizer = torch.optim.SGD(self.parameters(),0.3)
# #         self.loss = nn.MSELoss()
#         self.device = torch.device("cpu")
        
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.sigmoid(self.fc3(x))
#         return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(89, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

net = Net()

In [177]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(1000):
    optimizer.zero_grad()
    output = net(X_train)
    loss = criterion(output, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch+1}/{1000}, Loss: {loss.item():.4f}")


Epoch 1/1000, Loss: 0.0159
Epoch 51/1000, Loss: 0.0087
Epoch 101/1000, Loss: 0.0058
Epoch 151/1000, Loss: 0.0041
Epoch 201/1000, Loss: 0.0030
Epoch 251/1000, Loss: 0.0022
Epoch 301/1000, Loss: 0.0017
Epoch 351/1000, Loss: 0.0013
Epoch 401/1000, Loss: 0.0011
Epoch 451/1000, Loss: 0.0008
Epoch 501/1000, Loss: 0.0006
Epoch 551/1000, Loss: 0.0003
Epoch 601/1000, Loss: 0.0001
Epoch 651/1000, Loss: 0.0000
Epoch 701/1000, Loss: 0.0000
Epoch 751/1000, Loss: 0.0000
Epoch 801/1000, Loss: 0.0000
Epoch 851/1000, Loss: 0.0000
Epoch 901/1000, Loss: 0.0000
Epoch 951/1000, Loss: 0.0000


In [178]:
with torch.no_grad():
    output = net(X_test)
    predictions = (output > 0.5).float()
    accuracy = (predictions == y_test.view(-1, 1)).float().mean()
    print(f"Accuracy: {accuracy.item():.4f}")


Accuracy: 1.0000


In [180]:
torch.save(model.state_dict(), 'model.pt')

In [195]:
# Load the saved model
model = Net()
model.load_state_dict(torch.load("model.pt"))

# Put the model in evaluation mode
model.eval()

# Use the model to make predictions on new data
#input = [2021, 39.3, 88.5, 0.444, 13.8, 40.6, 0.339, 25.5, 47.9, 0.532, 16.5, 22.3, 0.74, 9.3, 33.3, 42.6, 23.6, 7.6, 5, 14.7, 19.5, 108.8, 42.7, 89, 0.48, 13.4, 35.2, 0.382, 29.3, 53.9, 0.544, 17.9, 22.8, 0.783, 9.9, 37.7, 47.6, 25.8, 8.5, 5.3, 14.7, 19.3, 116.7, 26.5, 17, 55, 20, 52, -7.9, 0.4, -7.5, 107.1, 114.9, -7.8, 101.4, 0.252, 0.459, 0.553, 0.521, 13, 19.8, 0.187, 0.555, 12.9, 77.1, 0.201, False, 30, 0.236111111, 0.25, 0.222222222, 0.323529412, 0.157894737, 0.428571429, 0.177777778, 0, 0, 0.5, 0.25, False, 0.3125, 1.375, 2.375, 2.625, 0, 0, 0, 0, 0, 0]
new_data = torch.tensor([39.3, 88.5, 100, 13.8, -40.6, 0.339, 0, 0, 0.532, 16.5, 22.3, 0.74, -9.3, -33.3, -42.6, -23.6, 7.6, 5, 14.7, 0, 108.8, 42.7, 89, 0.48, 13.4, 0, 0.382, 29.3, 0, 0.544, 17.9, 22.8, 0.783, 9.9, 37.7, 47.6, 25.8, 8.5, 5.3, 14.7, 19.3, 116.7, 26.5, 17, 55, 20, 52, -7.9, 0.4, -7.5, 107.1, 114.9, -7.8, 101.4, 0.252, 0.459, 0.553, 0.521, 13, 19.8, 0.187, 0.555, 12.9, 77.1, 0.201, False, 30, 0.236111111, 0.25, 0.222222222, 0.323529412, 0.157894737, 0.428571429, 0.177777778, 0, 0, 0.5, 0.25, False, 0.3125, 1.375, 2.375, 2.625, 0, 0, 0, 0, 0, 0])
output = model(new_data)

for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0745,  0.0355,  0.0284,  ..., -0.0910, -0.0211,  0.0005],
        [-0.0074,  0.1002, -0.0375,  ...,  0.0559,  0.0405, -0.0742],
        [ 0.0160,  0.0062,  0.0026,  ...,  0.0298, -0.0648, -0.0387],
        ...,
        [ 0.0896, -0.0804,  0.0363,  ..., -0.1030, -0.0590,  0.0319],
        [ 0.0397,  0.0838,  0.0240,  ...,  0.0394, -0.0725,  0.0628],
        [ 0.0068, -0.0457, -0.0338,  ..., -0.0620,  0.0577, -0.0389]],
       requires_grad=True)
Parameter containing:
tensor([ 6.7995e-02, -8.7668e-02,  1.0141e-01, -6.0563e-02, -3.9870e-02,
         6.7667e-02,  8.1813e-02, -3.6084e-02, -5.9940e-02, -9.9603e-02,
        -8.8033e-02,  3.5419e-02,  7.0659e-02,  7.8167e-02, -4.8926e-02,
        -1.9692e-02,  4.7587e-05,  5.7349e-02,  2.1920e-02, -6.0227e-02,
        -6.9172e-02, -7.5796e-02, -5.5496e-02, -6.4445e-02, -6.7224e-02,
         1.0398e-01, -4.4453e-02, -8.8217e-02, -6.0865e-02,  5.7918e-02,
         9.1616e-02, -7.4160e-02], requires_grad=True)
Pa

In [197]:
state_dict = model.state_dict()

print(state_dict['weight'])
print(state_dict['bias'])

<generator object Module.parameters at 0x0000025C793A3040>
